In [10]:
import os
import cv2
import random
import shutil
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras as keras
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from PIL import Image
import numpy as np
import pandas as pd

import os
for dirname, _, filenames in os.walk('/data'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
train_dir = 'data/Training'
test_dir = 'data/Testing'
classes = os.listdir(train_dir)
resize_size = 256
crop_size = 224

def preprocess_image(image):
    image = tf.image.resize(image, [resize_size, resize_size], method=tf.image.ResizeMethod.BILINEAR) #크기 조절
    image = tf.image.central_crop(image, central_fraction=crop_size / resize_size) #중앙 224x224
    image = tf.math.divide(image, 255.0) #normalize
    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]
    image = (image - mean) / std #다 normalize

    return image

from sklearn.preprocessing import LabelEncoder

X = [] #Image
y = [] #class
for i in classes:
    for data_dir in [train_dir, test_dir]:
        folderPath = os.path.join(data_dir,i)
        for j in tqdm(os.listdir(folderPath)):
            img = cv2.imread(os.path.join(folderPath,j)) #이미지 읽기
            img = preprocess_image(img) #전처리
            X.append(img) #X list 넣고
            y.append(i) # y list
X = np.array(X)
y = np.array(y)
y = tf.keras.utils.to_categorical([classes.index(label) for label in y]) #문자열 -> [0,3]

X_train, xx, y_train, yy = train_test_split(X,y, test_size=0.2, random_state=42) #training 분할 (train,val)
X_train, X_val, y_train, y_val = train_test_split(xx,yy, test_size=0.5, random_state=42) #training 분할 (train,val)

100%|██████████████████████████████████████████| 74/74 [00:00<00:00, 161.77it/s]


In [11]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16, ResNet50
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, CSVLogger
import os


epochs=50

version = "Final_VGG_16_1"
def create_model(base_model, output_size, model_name):
    model_dir = f"{version}/{model_name}/"
    for layer in base_model.layers:
        layer.trainable = False
    x = Flatten()(base_model.output)
    x = Dense(256, activation='relu')(x)
    x = Dropout(rate=0.3)(x)
    
    output = Dense(output_size, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=output)
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    if not os.path.exists(model_dir):
        os.makedirs(model_dir)
    early_stopping = EarlyStopping(monitor='val_loss', patience=100, verbose=1, restore_best_weights=True)
    best_model_checkpoint = ModelCheckpoint(f"{model_dir}best_model.h5", monitor='val_loss', save_best_only=True, mode='min', verbose=1)
    csv_logger = CSVLogger(f'training_log_{model_name}.csv', separator=',', append=False)
    return model, [early_stopping, best_model_checkpoint, csv_logger]


base_model_vgg = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
model_vgg, callbacks_vgg = create_model(base_model_vgg, output_size=4, model_name='vgg')

#base_model_resnet = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
#model_resnet, callbacks_resnet = create_model_ver1(base_model_resnet, output_size=4, model_name='resnet')

history_vgg = model_vgg.fit(X_train, y_train,
                            epochs=epochs,
                            validation_data=(X_val, y_val),
                            verbose=1,
                            callbacks=callbacks_vgg)

from sklearn.metrics import classification_report,accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

for model_name in ["vgg"]:
    best_model = keras.models.load_model(version+"/"+model_name+"/best_model.h5")
    y_pred = best_model.predict(X_test)
    y_pred_single_label = np.argmax(y_pred, axis=1)
    y_test_single_label = np.argmax(y_test, axis=1)
    precision = precision_score(y_test_single_label, y_pred_single_label, average='macro')
    recall = recall_score(y_test_single_label, y_pred_single_label, average='macro')
    f1 = f1_score(y_test_single_label, y_pred_single_label, average='macro')
    accuracy = accuracy_score(y_test_single_label, y_pred_single_label)
    precision = round(precision, 3)
    recall = round(recall, 3)
    f1 = round(f1, 3)
    accuracy = round(accuracy, 3)
    print(str(model_name))
    print(precision, recall, f1, accuracy)

Epoch 1/50
83/83 [==============================] - ETA: 0s - loss: 1.3277 - accuracy: 0.4828
Epoch 1: val_loss improved from inf to 0.84196, saving model to Final_VGG_16_1/vgg/best_model.h5
83/83 [==============================] - 26s 306ms/step - loss: 1.3277 - accuracy: 0.4828 - val_loss: 0.8420 - val_accuracy: 0.6531
Epoch 2/50
83/83 [==============================] - ETA: 0s - loss: 0.8270 - accuracy: 0.6617
Epoch 2: val_loss improved from 0.84196 to 0.77744, saving model to Final_VGG_16_1/vgg/best_model.h5
83/83 [==============================] - 25s 298ms/step - loss: 0.8270 - accuracy: 0.6617 - val_loss: 0.7774 - val_accuracy: 0.6667
Epoch 3/50
83/83 [==============================] - ETA: 0s - loss: 0.6468 - accuracy: 0.7510
Epoch 3: val_loss improved from 0.77744 to 0.64591, saving model to Final_VGG_16_1/vgg/best_model.h5
83/83 [==============================] - 25s 299ms/step - loss: 0.6468 - accuracy: 0.7510 - val_loss: 0.6459 - val_accuracy: 0.7755
Epoch 4/50
83/83 [=====

83/83 [==============================] - 25s 298ms/step - loss: 0.1430 - accuracy: 0.9508 - val_loss: 0.4403 - val_accuracy: 0.8299
Epoch 29/50
83/83 [==============================] - ETA: 0s - loss: 0.1768 - accuracy: 0.9383
Epoch 29: val_loss did not improve from 0.18496
83/83 [==============================] - 25s 297ms/step - loss: 0.1768 - accuracy: 0.9383 - val_loss: 0.2202 - val_accuracy: 0.9082
Epoch 30/50
83/83 [==============================] - ETA: 0s - loss: 0.1497 - accuracy: 0.9508
Epoch 30: val_loss did not improve from 0.18496
83/83 [==============================] - 25s 297ms/step - loss: 0.1497 - accuracy: 0.9508 - val_loss: 0.9236 - val_accuracy: 0.7313
Epoch 31/50
83/83 [==============================] - ETA: 0s - loss: 0.1295 - accuracy: 0.9576
Epoch 31: val_loss did not improve from 0.18496
83/83 [==============================] - 25s 297ms/step - loss: 0.1295 - accuracy: 0.9576 - val_loss: 0.2520 - val_accuracy: 0.8946
Epoch 32/50
83/83 [========================

In [12]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16, ResNet50
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, CSVLogger
import os

epochs=50

version = "Final_VGG_16_2"
def create_model(base_model, output_size, model_name):
    model_dir = f"{version}/{model_name}/"
    for layer in base_model.layers:
        layer.trainable = False
    x = Flatten()(base_model.output)
    x = Dense(256, activation='relu')(x)
    
    output = Dense(output_size, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=output)
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    if not os.path.exists(model_dir):
        os.makedirs(model_dir)
    early_stopping = EarlyStopping(monitor='val_loss', patience=100, verbose=1, restore_best_weights=True)
    best_model_checkpoint = ModelCheckpoint(f"{model_dir}best_model.h5", monitor='val_loss', save_best_only=True, mode='min', verbose=1)
    csv_logger = CSVLogger(f'training_log_{model_name}.csv', separator=',', append=False)
    return model, [early_stopping, best_model_checkpoint, csv_logger]


base_model_vgg = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
model_vgg, callbacks_vgg = create_model(base_model_vgg, output_size=4, model_name='vgg')

#base_model_resnet = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
#model_resnet, callbacks_resnet = create_model_ver1(base_model_resnet, output_size=4, model_name='resnet')

history_vgg = model_vgg.fit(X_train, y_train,
                            epochs=epochs,
                            validation_data=(X_val, y_val),
                            verbose=1,
                            callbacks=callbacks_vgg)

from sklearn.metrics import classification_report,accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

for model_name in ["vgg"]:
    best_model = keras.models.load_model(version+"/"+model_name+"/best_model.h5")
    y_pred = best_model.predict(X_test)
    y_pred_single_label = np.argmax(y_pred, axis=1)
    y_test_single_label = np.argmax(y_test, axis=1)
    precision = precision_score(y_test_single_label, y_pred_single_label, average='macro')
    recall = recall_score(y_test_single_label, y_pred_single_label, average='macro')
    f1 = f1_score(y_test_single_label, y_pred_single_label, average='macro')
    accuracy = accuracy_score(y_test_single_label, y_pred_single_label)
    precision = round(precision, 3)
    recall = round(recall, 3)
    f1 = round(f1, 3)
    accuracy = round(accuracy, 3)
    print(str(model_name))
    print(precision, recall, f1, accuracy)

Epoch 1/50
83/83 [==============================] - ETA: 0s - loss: 1.3613 - accuracy: 0.5002
Epoch 1: val_loss improved from inf to 0.97680, saving model to Final_VGG_16_2/vgg/best_model.h5
83/83 [==============================] - 27s 304ms/step - loss: 1.3613 - accuracy: 0.5002 - val_loss: 0.9768 - val_accuracy: 0.5374
Epoch 2/50
83/83 [==============================] - ETA: 0s - loss: 0.7350 - accuracy: 0.7098
Epoch 2: val_loss improved from 0.97680 to 0.72490, saving model to Final_VGG_16_2/vgg/best_model.h5
83/83 [==============================] - 25s 299ms/step - loss: 0.7350 - accuracy: 0.7098 - val_loss: 0.7249 - val_accuracy: 0.6871
Epoch 3/50
83/83 [==============================] - ETA: 0s - loss: 0.5811 - accuracy: 0.7794
Epoch 3: val_loss improved from 0.72490 to 0.66314, saving model to Final_VGG_16_2/vgg/best_model.h5
83/83 [==============================] - 25s 300ms/step - loss: 0.5811 - accuracy: 0.7794 - val_loss: 0.6631 - val_accuracy: 0.7347
Epoch 4/50
83/83 [=====

Epoch 29/50
83/83 [==============================] - ETA: 0s - loss: 0.5578 - accuracy: 0.7964
Epoch 29: val_loss did not improve from 0.17477
83/83 [==============================] - 26s 310ms/step - loss: 0.5578 - accuracy: 0.7964 - val_loss: 0.7332 - val_accuracy: 0.6939
Epoch 30/50
83/83 [==============================] - ETA: 0s - loss: 0.4312 - accuracy: 0.8411
Epoch 30: val_loss did not improve from 0.17477
83/83 [==============================] - 25s 296ms/step - loss: 0.4312 - accuracy: 0.8411 - val_loss: 0.3572 - val_accuracy: 0.8707
Epoch 31/50
83/83 [==============================] - ETA: 0s - loss: 0.2770 - accuracy: 0.9012
Epoch 31: val_loss did not improve from 0.17477
83/83 [==============================] - 25s 296ms/step - loss: 0.2770 - accuracy: 0.9012 - val_loss: 0.4300 - val_accuracy: 0.8129
Epoch 32/50
83/83 [==============================] - ETA: 0s - loss: 0.2190 - accuracy: 0.9247
Epoch 32: val_loss did not improve from 0.17477
83/83 [=========================

In [14]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16, ResNet50
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, CSVLogger
import os

epochs=50

version = "Final_ResNet_1"
def create_model(base_model, output_size, model_name):
    model_dir = f"{version}/{model_name}/"
    for layer in base_model.layers:
        layer.trainable = False
    x = Flatten()(base_model.output)
    x = Dense(256, activation='relu')(x)
    x = Dense(128, activation='relu')(x)
    x = Dense(64, activation='relu')(x)
    
    output = Dense(output_size, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=output)
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    if not os.path.exists(model_dir):
        os.makedirs(model_dir)
    early_stopping = EarlyStopping(monitor='val_loss', patience=100, verbose=1, restore_best_weights=True)
    best_model_checkpoint = ModelCheckpoint(f"{model_dir}best_model.h5", monitor='val_loss', save_best_only=True, mode='min', verbose=1)
    csv_logger = CSVLogger(f'training_log_{model_name}.csv', separator=',', append=False)
    return model, [early_stopping, best_model_checkpoint, csv_logger]


base_model_resnet = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
model_resnet, callbacks_resnet = create_model(base_model_resnet, output_size=4, model_name='resnet')

history_vgg = model_resnet.fit(X_train, y_train,
                            epochs=epochs,
                            validation_data=(X_val, y_val),
                            verbose=1,
                            callbacks=callbacks_resnet)

from sklearn.metrics import classification_report,accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

for model_name in ["resnet"]:
    best_model = keras.models.load_model(version+"/"+model_name+"/best_model.h5")
    y_pred = best_model.predict(X_test)
    y_pred_single_label = np.argmax(y_pred, axis=1)
    y_test_single_label = np.argmax(y_test, axis=1)
    precision = precision_score(y_test_single_label, y_pred_single_label, average='macro')
    recall = recall_score(y_test_single_label, y_pred_single_label, average='macro')
    f1 = f1_score(y_test_single_label, y_pred_single_label, average='macro')
    accuracy = accuracy_score(y_test_single_label, y_pred_single_label)
    precision = round(precision, 3)
    recall = round(recall, 3)
    f1 = round(f1, 3)
    accuracy = round(accuracy, 3)
    print(str(model_name))
    print(precision, recall, f1, accuracy)

Epoch 1/50
83/83 [==============================] - ETA: 0s - loss: 1.4671 - accuracy: 0.4828
Epoch 1: val_loss improved from inf to 0.99470, saving model to Final_ResNet_1/resnet/best_model.h5
83/83 [==============================] - 20s 225ms/step - loss: 1.4671 - accuracy: 0.4828 - val_loss: 0.9947 - val_accuracy: 0.5578
Epoch 2/50
83/83 [==============================] - ETA: 0s - loss: 0.8329 - accuracy: 0.6530
Epoch 2: val_loss improved from 0.99470 to 0.85494, saving model to Final_ResNet_1/resnet/best_model.h5
83/83 [==============================] - 16s 192ms/step - loss: 0.8329 - accuracy: 0.6530 - val_loss: 0.8549 - val_accuracy: 0.5850
Epoch 3/50
83/83 [==============================] - ETA: 0s - loss: 0.6774 - accuracy: 0.7181
Epoch 3: val_loss improved from 0.85494 to 0.63628, saving model to Final_ResNet_1/resnet/best_model.h5
83/83 [==============================] - 15s 184ms/step - loss: 0.6774 - accuracy: 0.7181 - val_loss: 0.6363 - val_accuracy: 0.7313
Epoch 4/50
83/

Epoch 29/50
83/83 [==============================] - ETA: 0s - loss: 0.0538 - accuracy: 0.9845
Epoch 29: val_loss did not improve from 0.21662
83/83 [==============================] - 15s 177ms/step - loss: 0.0538 - accuracy: 0.9845 - val_loss: 0.2176 - val_accuracy: 0.9184
Epoch 30/50
83/83 [==============================] - ETA: 0s - loss: 0.0272 - accuracy: 0.9951
Epoch 30: val_loss did not improve from 0.21662
83/83 [==============================] - 15s 177ms/step - loss: 0.0272 - accuracy: 0.9951 - val_loss: 0.3670 - val_accuracy: 0.8707
Epoch 31/50
83/83 [==============================] - ETA: 0s - loss: 0.0734 - accuracy: 0.9792
Epoch 31: val_loss did not improve from 0.21662
83/83 [==============================] - 15s 177ms/step - loss: 0.0734 - accuracy: 0.9792 - val_loss: 0.3228 - val_accuracy: 0.8844
Epoch 32/50
83/83 [==============================] - ETA: 0s - loss: 0.0630 - accuracy: 0.9837
Epoch 32: val_loss did not improve from 0.21662
83/83 [=========================

In [15]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16, ResNet50
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, CSVLogger
import os

epochs=50

version = "Final_ResNet_2"
def create_model(base_model, output_size, model_name):
    model_dir = f"{version}/{model_name}/"
    for layer in base_model.layers:
        layer.trainable = False
    x = Flatten()(base_model.output)
    x = Dense(256, activation='relu')(x)
    x = Dense(128, activation='relu')(x)
    x = Dense(64, activation='relu')(x)
    x = Dropout(rate=0.3)(x)
    
    output = Dense(output_size, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=output)
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    if not os.path.exists(model_dir):
        os.makedirs(model_dir)
    early_stopping = EarlyStopping(monitor='val_loss', patience=100, verbose=1, restore_best_weights=True)
    best_model_checkpoint = ModelCheckpoint(f"{model_dir}best_model.h5", monitor='val_loss', save_best_only=True, mode='min', verbose=1)
    csv_logger = CSVLogger(f'training_log_{model_name}.csv', separator=',', append=False)
    return model, [early_stopping, best_model_checkpoint, csv_logger]


base_model_resnet = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
model_resnet, callbacks_resnet = create_model(base_model_resnet, output_size=4, model_name='resnet')

history_vgg = model_resnet.fit(X_train, y_train,
                            epochs=epochs,
                            validation_data=(X_val, y_val),
                            verbose=1,
                            callbacks=callbacks_resnet)

from sklearn.metrics import classification_report,accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

for model_name in ["resnet"]:
    best_model = keras.models.load_model(version+"/"+model_name+"/best_model.h5")
    y_pred = best_model.predict(X_test)
    y_pred_single_label = np.argmax(y_pred, axis=1)
    y_test_single_label = np.argmax(y_test, axis=1)
    precision = precision_score(y_test_single_label, y_pred_single_label, average='macro')
    recall = recall_score(y_test_single_label, y_pred_single_label, average='macro')
    f1 = f1_score(y_test_single_label, y_pred_single_label, average='macro')
    accuracy = accuracy_score(y_test_single_label, y_pred_single_label)
    precision = round(precision, 3)
    recall = round(recall, 3)
    f1 = round(f1, 3)
    accuracy = round(accuracy, 3)
    print(str(model_name))
    print(precision, recall, f1, accuracy)

Epoch 1/50
83/83 [==============================] - ETA: 0s - loss: 1.3866 - accuracy: 0.4514
Epoch 1: val_loss improved from inf to 1.06777, saving model to Final_ResNet_2/resnet/best_model.h5
83/83 [==============================] - 22s 227ms/step - loss: 1.3866 - accuracy: 0.4514 - val_loss: 1.0678 - val_accuracy: 0.5102
Epoch 2/50
83/83 [==============================] - ETA: 0s - loss: 0.8871 - accuracy: 0.6258
Epoch 2: val_loss improved from 1.06777 to 1.02223, saving model to Final_ResNet_2/resnet/best_model.h5
83/83 [==============================] - 16s 190ms/step - loss: 0.8871 - accuracy: 0.6258 - val_loss: 1.0222 - val_accuracy: 0.5850
Epoch 3/50
83/83 [==============================] - ETA: 0s - loss: 0.7352 - accuracy: 0.7003
Epoch 3: val_loss improved from 1.02223 to 0.69933, saving model to Final_ResNet_2/resnet/best_model.h5
83/83 [==============================] - 15s 182ms/step - loss: 0.7352 - accuracy: 0.7003 - val_loss: 0.6993 - val_accuracy: 0.7109
Epoch 4/50
83/

Epoch 28/50
83/83 [==============================] - ETA: 0s - loss: 0.2079 - accuracy: 0.9353
Epoch 28: val_loss did not improve from 0.21616
83/83 [==============================] - 15s 177ms/step - loss: 0.2079 - accuracy: 0.9353 - val_loss: 0.3568 - val_accuracy: 0.8980
Epoch 29/50
83/83 [==============================] - ETA: 0s - loss: 0.0892 - accuracy: 0.9712
Epoch 29: val_loss did not improve from 0.21616
83/83 [==============================] - 15s 175ms/step - loss: 0.0892 - accuracy: 0.9712 - val_loss: 0.2670 - val_accuracy: 0.9150
Epoch 30/50
83/83 [==============================] - ETA: 0s - loss: 0.0969 - accuracy: 0.9690
Epoch 30: val_loss did not improve from 0.21616
83/83 [==============================] - 15s 175ms/step - loss: 0.0969 - accuracy: 0.9690 - val_loss: 0.2999 - val_accuracy: 0.9014
Epoch 31/50
83/83 [==============================] - ETA: 0s - loss: 0.0934 - accuracy: 0.9743
Epoch 31: val_loss did not improve from 0.21616
83/83 [=========================